In [5]:
import numpy as np
import pandas as pd
import plotly as py
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.figure_factory import create_distplot
import statsmodels.formula.api as smf
from statsmodels.graphics.gofplots import ProbPlot
import missingno as msno
import json
from pandas.io.json import json_normalize
py.offline.init_notebook_mode(connected=True)
%matplotlib inline

In [71]:
import json
with open('./Datasets/flight-results-100first.json') as json_data:
    d = json.load(json_data)

In [109]:
df = pd.DataFrame()
for i in range(len(d)):
    df = df.append(json_normalize(d[i]), ignore_index = True)


In [112]:
df.isnull().sum()




airline            0
arrival            0
date               0
departure          0
flight duration    0
plane              0
plane code         0
stops              0
ticket price       0
timings            0
dtype: int64

In [113]:
df.columns

Index(['airline', 'arrival', 'date', 'departure', 'flight duration', 'plane',
       'plane code', 'stops', 'ticket price', 'timings'],
      dtype='object')